In [ ]:
!pip install tensorflow pandas numpy scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 789.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 144.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 128.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.6 MB/s eta 0:00:00


In [ ]:
# -*- coding: utf-8 -*-
"""
Este script crea, entrena y evalúa un modelo de red neuronal para
predecir el resultado de la multiplicación de matrices 2x2.
Está basado en la estructura de un modelo para predicción de ratings de películas.

Uso:
1. Asegúrate de tener las bibliotecas necesarias instaladas:
   pip install tensorflow pandas numpy scikit-learn
2. Reemplaza la URL del dataset de ejemplo por la URL de tu propio dataset en GitHub.
3. Ejecuta el script.
"""

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import io
import requests

# --- 1. Carga y Preparación de Datos ---

def generate_and_get_sample_data():
    """Genera un dataset de ejemplo si la URL no está disponible o para demostración."""
    print("Generando un dataset de ejemplo...")
    num_samples = 1000
    data = []
    for _ in range(num_samples):
        # Generar dos matrices 2x2 aleatorias
        matrix_a = np.random.rand(2, 2) * 10  # Valores entre 0 y 10
        matrix_b = np.random.rand(2, 2) * 10

        # Calcular el resultado de la multiplicación
        result_matrix = np.dot(matrix_a, matrix_b)

        # Aplanar las matrices para el formato CSV
        row = list(matrix_a.flatten()) + list(matrix_b.flatten()) + list(result_matrix.flatten())
        data.append(row)

    columns = ['a11', 'a12', 'a21', 'a22', 'b11', 'b12', 'b21', 'b22', 'c11', 'c12', 'c21', 'c22']
    df = pd.DataFrame(data, columns=columns)

    # Convertir el DataFrame a un string en formato CSV para simular la descarga
    csv_string = df.to_csv(index=False)
    return io.StringIO(csv_string)


GITHUB_URL = 'https://raw.githubusercontent.com/yoromero1/Tarea_Machine_Learning/refs/heads/main/matrix_multiplication_dataset.csv'

try:
    if GITHUB_URL:
        print(f"Descargando dataset desde: {GITHUB_URL}")
        response = requests.get(GITHUB_URL)
        response.raise_for_status() # Lanza un error si la petición falla
        data_source = io.StringIO(response.text)
        df = pd.read_csv(data_source)
    else:
        raise ValueError("URL vacía")
except (requests.exceptions.RequestException, ValueError) as e:
    print(f"No se pudo descargar el dataset desde la URL. Causa: {e}")
    print("Usando dataset de ejemplo generado localmente.")
    data_source = generate_and_get_sample_data()
    df = pd.read_csv(data_source)

print("\n--- Primeras 5 filas del dataset: ---")
print(df.head())

# Separar las características (entradas) de las etiquetas (salidas)
# Entradas (X): Los 8 valores de las dos matrices a multiplicar.
features = ['a11', 'a12', 'a21', 'a22', 'b11', 'b12', 'b21', 'b22']
# Salidas (y): Los 4 valores de la matriz resultado.
labels = ['c11', 'c12', 'c21', 'c22']

X = df[features].values
y = df[labels].values

# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características. Esto ayuda a que el modelo converja más rápido.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"\nDatos listos: {X_train.shape[0]} muestras para entrenamiento, {X_test.shape[0]} para prueba.")


# --- 2. Construcción del Modelo de Red Neuronal ---
# Esta sección es análoga a la creación del modelo con Keras.
# Hemos definido una red neuronal secuencial con varias capas densas.

# El input tiene 8 neuronas (una por cada valor de las matrices de entrada).
# El output tiene 4 neuronas (una por cada valor de la matriz resultado).
model = tf.keras.Sequential([
    # Capa de entrada con 8 neuronas
    tf.keras.layers.Input(shape=(8,)),
    # Primera capa oculta con 64 neuronas y activación ReLU
    tf.keras.layers.Dense(64, activation='relu'),
    # Segunda capa oculta con 64 neuronas
    tf.keras.layers.Dense(64, activation='relu'),
    # Tercera capa oculta con 32 neuronas
    tf.keras.layers.Dense(32, activation='relu'),
    # Capa de salida con 4 neuronas (sin función de activación, o 'linear')
    # ya que es un problema de regresión (predecir valores continuos).
    tf.keras.layers.Dense(4)
])

# --- 3. Compilación del Modelo ---
# Aquí definimos el optimizador y la función de pérdida.
# Usamos 'mean_squared_error' porque queremos minimizar la diferencia
# cuadrática entre los valores predichos y los reales.
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_absolute_error']) # Métrica para monitorear

model.summary()


# --- 4. Entrenamiento del Modelo ---
# Entrenamos el modelo con los datos de entrenamiento.
# 'epochs' es el número de veces que el modelo verá todo el dataset.
# 'validation_data' permite evaluar el rendimiento en el conjunto de prueba
# en cada época.
print("\n--- Iniciando entrenamiento del modelo... ---")
history = model.fit(
    X_train,
    y_train,
    epochs=300,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1 # Muestra una barra de progreso
)
print("--- Entrenamiento finalizado. ---")


# --- 5. Evaluación del Modelo ---
# Evaluamos el rendimiento final del modelo con los datos de prueba.
print("\n--- Evaluando el modelo con datos de prueba... ---")
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Pérdida (Mean Squared Error) en el conjunto de prueba: {loss:.4f}")
print(f"Error Absoluto Medio en el conjunto de prueba: {mae:.4f}")


# --- 6. Realizar Predicciones ---
# Ahora usamos el modelo entrenado para predecir el resultado de una nueva
# multiplicación de matrices.

print("\n======= Realizando las predicciones... =========")

# Definimos las matrices 2x2 para multiplicar
print("\n----- Matriz 1 -------")
matrix_a_example = np.array([[2, 3],
                             [4, 5]])

matrix_b_example = np.array([[6, 1],
                             [7, 8]])

#------------
print("\n----- Matriz 2 ----------")
matrix_a2_example = np.random.randint(low = -20, high = 20, size=(2, 2))
matrix_b2_example = np.random.randint(low = -20, high = 20, size=(2, 2))

#---------
print("\n----- Matriz 3 ----------")
matrix_a3_example = np.random.randint(low = -20, high = 20, size=(2, 2))
matrix_b3_example = np.random.randint(low = -20, high = 20, size=(2, 2))

#---------
print("\n----- Matriz 4 ----------")
matrix_a4_example = np.random.randint(low = -20, high = 20, size=(2, 2))
matrix_b4_example = np.random.randint(low = -20, high = 20, size=(2, 2))

#---------
print("\n----- Matriz 5 ----------")
matrix_a5_example = np.random.randint(low = -20, high = 20, size=(2, 2))
matrix_b5_example = np.random.randint(low = -20, high = 20, size=(2, 2))

#---------
print("\n----- Matriz 6 ----------")
matrix_a6_example = np.random.randint(low = -20, high = 20, size=(2, 2))
matrix_b6_example = np.random.randint(low = -20, high = 20, size=(2, 2))

#---------
print("\n----- Matriz 7 ----------")
matrix_a7_example = np.random.randint(low = -20, high = 20, size=(2, 2))
matrix_b7_example = np.random.randint(low = -20, high = 20, size=(2, 2))

#---------
print("\n----- Matriz 8 ----------")
matrix_a8_example = np.random.randint(low = -20, high = 20, size=(2, 2))
matrix_b8_example = np.random.randint(low = -20, high = 20, size=(2, 2))

#---------
print("\n----- Matriz 9 ----------")
matrix_a9_example = np.random.randint(low = -20, high = 20, size=(2, 2))
matrix_b9_example = np.random.randint(low = -20, high = 20, size=(2, 2))

#---------
print("\n----- Matriz 10 ----------")
matrix_a10_example = np.random.randint(low = -20, high = 20, size=(2, 2))
matrix_b10_example = np.random.randint(low = -20, high = 20, size=(2, 2))

# Calculamos el resultado real para poder comparar
real_result = np.dot(matrix_a_example, matrix_b_example)
real_result_2 = np.dot(matrix_a2_example, matrix_b2_example)
real_result_3 = np.dot(matrix_a3_example, matrix_b3_example)
real_result_4 = np.dot(matrix_a4_example, matrix_b4_example)
real_result_5 = np.dot(matrix_a5_example, matrix_b5_example)
real_result_6 = np.dot(matrix_a6_example, matrix_b6_example)
real_result_7 = np.dot(matrix_a7_example, matrix_b7_example)
real_result_8 = np.dot(matrix_a8_example, matrix_b8_example)
real_result_9 = np.dot(matrix_a9_example, matrix_b9_example)
real_result_10 = np.dot(matrix_a10_example, matrix_b10_example)

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

# Preparamos los datos de entrada para el modelo
input_vector = np.concatenate([matrix_a_example.flatten(), matrix_b_example.flatten()]) # 1. Aplanar las matrices a un solo vector de 8 elementos
input_vector_reshaped = input_vector.reshape(1, -1) # 2. Reorganizarlo para que tenga la forma (1, 8) que espera el modelo
input_vector_scaled = scaler.transform(input_vector_reshaped) # 3. Escalar los datos con el mismo 'scaler' que usamos para el entrenamiento

input_vector2 = np.concatenate([matrix_a2_example.flatten(), matrix_b2_example.flatten()])
input_vector_reshaped2 = input_vector2.reshape(1, -1)
input_vector_scaled2 = scaler.transform(input_vector_reshaped2)

input_vector3 = np.concatenate([matrix_a3_example.flatten(), matrix_b3_example.flatten()])
input_vector_reshaped3 = input_vector3.reshape(1, -1)
input_vector_scaled3 = scaler.transform(input_vector_reshaped3)

input_vector4 = np.concatenate([matrix_a4_example.flatten(), matrix_b4_example.flatten()])
input_vector_reshaped4 = input_vector4.reshape(1, -1)
input_vector_scaled4 = scaler.transform(input_vector_reshaped4)

input_vector5 = np.concatenate([matrix_a5_example.flatten(), matrix_b5_example.flatten()])
input_vector_reshaped5 = input_vector5.reshape(1, -1)
input_vector_scaled5 = scaler.transform(input_vector_reshaped5)

input_vector6 = np.concatenate([matrix_a6_example.flatten(), matrix_b6_example.flatten()])
input_vector_reshaped6 = input_vector6.reshape(1, -1)
input_vector_scaled6 = scaler.transform(input_vector_reshaped6)

input_vector7 = np.concatenate([matrix_a7_example.flatten(), matrix_b7_example.flatten()])
input_vector_reshaped7 = input_vector7.reshape(1, -1)
input_vector_scaled7 = scaler.transform(input_vector_reshaped7)

input_vector8 = np.concatenate([matrix_a8_example.flatten(), matrix_b8_example.flatten()])
input_vector_reshaped8 = input_vector8.reshape(1, -1)
input_vector_scaled8 = scaler.transform(input_vector_reshaped8)

input_vector9 = np.concatenate([matrix_a9_example.flatten(), matrix_b9_example.flatten()])
input_vector_reshaped9 = input_vector9.reshape(1, -1)
input_vector_scaled9 = scaler.transform(input_vector_reshaped9)

inpurt_vector10 = np.concatenate([matrix_a10_example.flatten(), matrix_b10_example.flatten()])
input_vector_reshaped10 = inpurt_vector10.reshape(1, -1)
input_vector_scaled10 = scaler.transform(input_vector_reshaped10)

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

# Realizamos la predicción
predicted_result_vector = model.predict(input_vector_scaled)
predicted_result_vector2 = model.predict(input_vector_scaled2)
predicted_result_vector3 = model.predict(input_vector_scaled3)
predicted_result_vector4 = model.predict(input_vector_scaled4)
predicted_result_vector5 = model.predict(input_vector_scaled5)
predicted_result_vector6 = model.predict(input_vector_scaled6)
predicted_result_vector7 = model.predict(input_vector_scaled7)
predicted_result_vector8 = model.predict(input_vector_scaled8)
predicted_result_vector9 = model.predict(input_vector_scaled9)
predicted_result_vector10 = model.predict(input_vector_scaled10)

# Reorganizamos el vector de salida a una matriz 2x2
predicted_matrix = predicted_result_vector.reshape(2, 2)
predicted_matrix2 = predicted_result_vector2.reshape(2, 2)
predicted_matrix3 = predicted_result_vector3.reshape(2, 2)
predicted_matrix4 = predicted_result_vector4.reshape(2, 2)
predicted_matrix5 = predicted_result_vector5.reshape(2, 2)
predicted_matrix6 = predicted_result_vector6.reshape(2, 2)
predicted_matrix7 = predicted_result_vector7.reshape(2, 2)
predicted_matrix8 = predicted_result_vector8.reshape(2, 2)
predicted_matrix9 = predicted_result_vector9.reshape(2, 2)
predicted_matrix10 = predicted_result_vector10.reshape(2, 2)

# Mostramos los resultados
print("\n--- Resultados de la matriz 1: ---")
print(f"\nMatriz A:\n{matrix_a_example}")
print(f"\nMatriz B:\n{matrix_b_example}")
print(f"\nResultado Real (Calculado con NumPy):\n{real_result}")
print(f"\nResultado Predicho por el Modelo:\n{predicted_matrix}")
print(f"\nDiferencia (Predicho - Real):\n{predicted_matrix - real_result}")

print("\n------")

print("\n----- Resultados de la Matriz 2 ------")
print(f"\nMatriz A:\n{matrix_a2_example}")
print(f"\nMatriz B:\n{matrix_b2_example}")
print(f"\nResultado Real (Calculado con NumPy):\n{real_result_2}")
print(f"\nResultado Predicho por el Modelo:\n{predicted_matrix2}")
print(f"\nDiferencia (Predicho - Real):\n{predicted_matrix - real_result}")

print("\n------")

print("\n---- Resultados de la matriz 3 ------------")
print(f"\nMatriz A:\n{matrix_a3_example}")
print(f"\nMatriz B:\n{matrix_b3_example}")
print(f"\nResultado Real (Calculado con NumPy):\n{real_result_3}")
print(f"\nResultado Predicho por el Modelo:\n{predicted_matrix3}")
print(f"\nDiferencia (Predicho - Real):\n{predicted_matrix - real_result}")

print("\n------")

print("\n---- Resultados de la matriz 4 ------------")
print(f"\nMatriz A:\n{matrix_a4_example}")
print(f"\nMatriz B:\n{matrix_b4_example}")
print(f"\nResultado Real (Calculado con NumPy):\n{real_result_4}")
print(f"\nResultado Predicho por el Modelo:\n{predicted_matrix4}")
print(f"\nDiferencia (Predicho - Real):\n{predicted_matrix - real_result}")

print("\n------")

print("\n---- Resultados de la matriz 5 ------------")
print(f"\nMatriz A:\n{matrix_a5_example}")
print(f"\nMatriz B:\n{matrix_b5_example}")
print(f"\nResultado Real (Calculado con NumPy):\n{real_result_5}")
print(f"\nResultado Predicho por el Modelo:\n{predicted_matrix5}")
print(f"\nDiferencia (Predicho - Real):\n{predicted_matrix - real_result}")

print("\n------")

print("\n---- Resultados de la matriz 6 ------------")
print(f"\nMatriz A:\n{matrix_a6_example}")
print(f"\nMatriz B:\n{matrix_b6_example}")
print(f"\nResultado Real (Calculado con NumPy):\n{real_result_6}")
print(f"\nResultado Predicho por el Modelo:\n{predicted_matrix6}")
print(f"\nDiferencia (Predicho - Real):\n{predicted_matrix - real_result}")

print("\n------")

print("\n---- Resultados de la matriz 7 ------------")
print(f"\nMatriz A:\n{matrix_a7_example}")
print(f"\nMatriz B:\n{matrix_b7_example}")
print(f"\nResultado Real (Calculado con NumPy):\n{real_result_7}")
print(f"\nResultado Predicho por el Modelo:\n{predicted_matrix7}")
print(f"\nDiferencia (Predicho - Real):\n{predicted_matrix - real_result}")

print("\n------")

print("\n---- Resultados de la matriz 8 ------------")
print(f"\nMatriz A:\n{matrix_a8_example}")
print(f"\nMatriz B:\n{matrix_b8_example}")
print(f"\nResultado Real (Calculado con NumPy):\n{real_result_8}")
print(f"\nResultado Predicho por el Modelo:\n{predicted_matrix8}")
print(f"\nDiferencia (Predicho - Real):\n{predicted_matrix - real_result}")

print("\n------")

print("\n---- Resultados de la matriz 9 ------------")
print(f"\nMatriz A:\n{matrix_a9_example}")
print(f"\nMatriz B:\n{matrix_b9_example}")
print(f"\nResultado Real (Calculado con NumPy):\n{real_result_9}")
print(f"\nResultado Predicho por el Modelo:\n{predicted_matrix9}")
print(f"\nDiferencia (Predicho - Real):\n{predicted_matrix - real_result}")

print("\n------")

print("\n---- Resultados de la matriz 10 ------------")
print(f"\nMatriz A:\n{matrix_a10_example}")
print(f"\nMatriz B:\n{matrix_b10_example}")
print(f"\nResultado Real (Calculado con NumPy):\n{real_result_10}")
print(f"\nResultado Predicho por el Modelo:\n{predicted_matrix10}")
print(f"\nDiferencia (Predicho - Real):\n{predicted_matrix - real_result}")


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


Descargando dataset desde: https://raw.githubusercontent.com/yoromero1/Tarea_Machine_Learning/refs/heads/main/matrix_multiplication_dataset.csv

--- Primeras 5 filas del dataset: ---
   a11  a12  a21  a22  b11  b12  b21  b22  c11  c12  c21  c22
0   17  -18    3  -16    2  -10  -15    7  304 -296  246 -142
1   -9    6    2    5    3   -7    0   12  -27  135    6   46
2   17   10   -3   12   -6   -8   18   -8   78 -216  234  -72
3  -15  -13  -10   -3  -15   13   13   -7   56 -104  111 -109
4   20  -11    9    2   18   -3   12    0  228  -60  186  -27

Datos listos: 40000 muestras para entrenamiento, 10000 para prueba.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,948 (27.14 KB)

 Trainable params: 6,948 (27.14 KB)

 Non-trainable params: 0 (0.00 B)


--- Iniciando entrenamiento del modelo... ---
Epoch 1/300
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 27080.5762 - mean_absolute_error: 120.7205 - val_loss: 1388.0176 - val_mean_absolute_error: 28.6904
Epoch 2/300
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1274.9368 - mean_absolute_error: 27.5464 - val_loss: 1149.5358 - val_mean_absolute_error: 26.1398
Epoch 3/300
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1112.7466 - mean_absolute_error: 25.8400 - val_loss: 1087.9139 - val_mean_absolute_error: 25.4888
Epoch 4/300
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1044.2173 - mean_absolute_error: 24.9464 - val_loss: 1031.6055 - val_mean_absolute_error: 24.7124
Epoch 5/300
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1008.3599 - mean_absolute_error: 24.5466 - val_loss: 1032.0389 - val_mean_absolute_error: 24.7973
Epoch 6/300
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 979.1227 - mean_absolute_error: 24.1927 - val_loss: 993.4867 - val_mean_absolute_er